In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from itertools import cycle 
a = 'ì..ìì...................'
b = '...................ì..ìì'
print(a)
print(b)



def shift(s, d):
    if d == +1:
        if s[-1] == 'ì':
            return s
        else:
            shf = s[len(s)-1:] + s[:len(s)-1]
            return shf
    if d == -1:
        if s[0] == 'ì':
            return s
        else:
            shf = s[1:] + s[:1] 
            return shf
il = 0
ir = 0
dl = None
dr = None
bb1 = b
bb2 = b
while bb1 != a:
    il += 1
    nbb1 = shift(bb1, 1)
    print(nbb1)
    if nbb1 == a: 
        dl = il
        break
    if bb1 == nbb1: 
        print('wall')
        break
    else: bb1 = nbb1
     
while bb2 != a:
    ir += 1
    nbb2 = shift(bb2, -1)
    print(nbb2)
    if nbb2 == a:
        dr = ir 
        break
    if bb2 == nbb2: 
        print('wall')
        break
    else: bb2 = nbb2
    
print(dl, dr)

dist = dl if dl != None else dr 

print('register', dist)
print(a+'+'+b[::-1])

ì..ìì...................
...................ì..ìì
...................ì..ìì
wall
..................ì..ìì.
.................ì..ìì..
................ì..ìì...
...............ì..ìì....
..............ì..ìì.....
.............ì..ìì......
............ì..ìì.......
...........ì..ìì........
..........ì..ìì.........
.........ì..ìì..........
........ì..ìì...........
.......ì..ìì............
......ì..ìì.............
.....ì..ìì..............
....ì..ìì...............
...ì..ìì................
..ì..ìì.................
.ì..ìì..................
ì..ìì...................
None 19
register 19
ì..ìì...................+ìì..ì...................


In [4]:
import sys
sys.path.append('..')

import numpy as np

from hdna import *

In [5]:
stru = Structure('(((((((((+)))))))))')
stru.register

0

In [6]:
M = Model(standard=True)
M.setgeometry(theta=90, phi=180)
A = Strand(M, 'ACACACACACAC')
B = A.complementary()
K = Kinetics(M, A, B)

In [32]:
N = Kinetwork(M, A, B)    
N.save_graph('.')

((((((((((..+)))))))))).. --> ((((((((((((+)))))))))))) inchworming duplex 2
((((((((....+)))))))).... --> ((((((((((((+)))))))))))) inchworming duplex 4
((((((......+))))))...... --> ((((((((((((+)))))))))))) inchworming duplex 6
((((........+))))........ --> ((((((((((((+)))))))))))) inchworming duplex 8
((..........+)).......... --> ((((((((((((+)))))))))))) inchworming duplex 10
..........((+..........)) --> ((((((((((((+)))))))))))) inchworming duplex 10
........((((+........)))) --> ((((((((((((+)))))))))))) inchworming duplex 8
......((((((+......)))))) --> ((((((((((((+)))))))))))) inchworming duplex 6
....((((((((+....)))))))) --> ((((((((((((+)))))))))))) inchworming duplex 4
..((((((((((+..)))))))))) --> ((((((((((((+)))))))))))) inchworming duplex 2
((((((((((..+)))))))))).. --> ((((((((....+)))))))).... inchworming 2
((((((((((..+)))))))))).. --> ((((((......+))))))...... inchworming 4
((((((((((..+)))))))))).. --> ((((........+))))........ inchworming 6
((((((((((..+)))))

In [8]:
N.connect_slidings()

((((((((((((+)))))))))))) ((((((((((..+)))))))))).. inchworming duplex 2
((((((((((((+)))))))))))) ((((((((....+)))))))).... inchworming duplex 4
((((((((((((+)))))))))))) ((((((......+))))))...... inchworming duplex 6
((((((((((((+)))))))))))) ((((........+))))........ inchworming duplex 8
((((((((((((+)))))))))))) ((..........+)).......... inchworming duplex 10
((((((((((((+)))))))))))) ..........((+..........)) inchworming duplex 10
((((((((((((+)))))))))))) ........((((+........)))) inchworming duplex 8
((((((((((((+)))))))))))) ......((((((+......)))))) inchworming duplex 6
((((((((((((+)))))))))))) ....((((((((+....)))))))) inchworming duplex 4
((((((((((((+)))))))))))) ..((((((((((+..)))))))))) inchworming duplex 2
((((((((((..+)))))))))).. ((((((((....+)))))))).... inchworming 2
((((((((((..+)))))))))).. ((((((......+))))))...... inchworming 4
((((((((((..+)))))))))).. ((((........+))))........ inchworming 6
((((((((((..+)))))))))).. ((..........+)).......... inchworming 8
((((

In [9]:
nodz = list(N.filternodes('state', lambda x: x == 'on_nucleation', N.DG).nodes())

objs = [N.DG.nodes[node]['obj'] for node in nodz]

In [10]:
for ob in objs:
    print(ob.structure)

(...........+...........)
.(..........+..........).
..(.........+.........)..
...(........+........)...
....(.......+.......)....
.....(......+......).....
......(.....+.....)......
.......(....+....).......
........(...+...)........
.........(..+..).........
..........(.+.)..........
...........(+)...........


In [11]:
def get_ix(string, char):
    indices = []
    for i, e in enumerate(list(string)):
        if e == char:
            indices.append(i)
    return indices 

In [12]:
on = 0
print(objs[on].structure)
l = objs[on].structure.split('+')[0]
r = objs[on].structure.split('+')[1][::-1]
ixl = get_ix(l,'(')[0]
ixr = get_ix(r,')')[0]

print(objs[on].s1.sdist[ixl])
objs[on].s1.sdist[ixl] == objs[on].s1.sdist[ixr]


(...........+...........)
0


True

In [13]:
np.ceil(2*SXGEO.PERSISTENCE/SXGEO.MONODIST)

7.0

In [14]:
print(f'{K.ksphere_sano():.3e}')

-6.350e+14


In [15]:
bottomheignt = 2*SXGEO.PERSISTENCE
topgyradius  = K.gyradiuses(N=4)
kc = K.ksphere_sano(a=topgyradius, badd=(bottomheignt))
print(f'{kc:.3e}')
kc = K.ksphere_sano(SXGEO.CY_RADIUS)
print(f'{kc:.3e}')
print()
print(SXGEO.CY_RADIUS)
print(K.gr1)

-5.373e+14
-5.091e+14

1e-07
2.3417326918331218e-07
